In [1]:
# !wget https://cdn.commonvoice.mozilla.org/cv-corpus-5.1-2020-06-22/id.tar.gz
# !tar -zxf id.tar.gz

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part3.tar
# !mkdir part1-v2
# !tar -xf semisupervised-26-02-2021-part3.tar -C part1-v2

In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part4.tar
# !mkdir part2-v2
# !tar -xf semisupervised-26-02-2021-part4.tar -C part2-v2

In [ ]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part1.tar
# !tar -xf semisupervised-24-03-2021-part1.tar

In [ ]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part2.tar
# !tar -xf semisupervised-24-03-2021-part2.tar

In [ ]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part3.tar
# !tar -xf semisupervised-24-03-2021-part3.tar

In [6]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/streaming.zip -O wikipedia-asr.zip
# !unzip wikipedia-asr.zip

In [7]:
# wget https://f000.backblazeb2.com/file/malaya-speech-model/data/news-speech.zip
# wget https://f000.backblazeb2.com/file/malaya-speech-model/collections/transcript-news.json
# unzip news-speech.zip -d news

In [8]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/trainset-audiobook.tar.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/text-audiobook.tar.gz
# !tar -xf trainset-audiobook.tar.gz
# !tar -zxf text-audiobook.tar.gz

In [9]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [10]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import json

In [11]:
base_directory = '/home/husein/speech-bahasa'

In [12]:
df = pd.read_csv(f'{base_directory}/cv-corpus-5.1-2020-06-22/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(7490, 10)


In [13]:
id_commonvoice = []
for i in range(len(df)):
    p = f"{base_directory}/cv-corpus-5.1-2020-06-22/id/clips/{df['path'].iloc[i]}"
    t = df['sentence'].iloc[i]
    if len(t) < 5:
        continue
    id_commonvoice.append((p, t))

len(id_commonvoice)

7490

In [14]:
malay = glob(f'{base_directory}/part*/output-wav/*.wav')
len(malay)

1008453

In [15]:
malay.extend(glob(f'{base_directory}/part*/semisupervised/output-wav/*.wav'))
len(malay)

1322836

In [16]:
malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 3:
            continue
        malays.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 1322836/1322836 [07:28<00:00, 2946.80it/s]


In [17]:
wikipedia = []
wavs = glob(f'{base_directory}/streaming/*wav')
for i in tqdm(wavs):
    text = os.path.split(i)[1].replace('.wav', '')
    wikipedia.append((i, text))
    
len(wikipedia)

100%|██████████| 2887/2887 [00:00<00:00, 247212.36it/s]


2887

In [18]:
news = []
wavs = glob(f'{base_directory}/news/audio/*wav')

with open(f'{base_directory}/transcript-news.json') as fopen:
    transcript_news = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript_news[int(index)]
    news.append((i, text))

100%|██████████| 2151/2151 [00:00<00:00, 29117.05it/s]


In [19]:
audiobook = []
wavs = glob(f'{base_directory}/combined/*wav')
for i in tqdm(wavs):
    t = '/'.join(i.split('<>')[1:])
    t = t.split('.wav')[0]
    t = t.replace('output-wav', 'output-text')
    with open(f'{base_directory}/text-audiobook/{t}.wav.txt') as fopen:
        text = fopen.read()
    audiobook.append((i, text))

100%|██████████| 64341/64341 [00:06<00:00, 10594.83it/s]


In [20]:
df = pd.read_csv(f'{base_directory}/haqkiem/metadata.csv', header = None, sep = '|')
txts = df.values.tolist()
haqkiem = []
for f in tqdm(txts):
    text = f[1]
    text = text.split('.,,')[0]
    f = f[0]
    r = f'{base_directory}/haqkiem/{f}.wav'
    haqkiem.append((r, text))

100%|██████████| 4294/4294 [00:00<00:00, 642331.80it/s]


In [21]:
# import IPython.display as ipd

# ipd.Audio(audiobook[0][0])

In [22]:
audios = id_commonvoice + malays + wikipedia + news + audiobook + haqkiem
audios, texts = zip(*audios)

In [23]:
len(texts)

1403509

In [24]:
import unicodedata
import re

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "'", "-", "z", "0", "1", "x", "2", "q", "*", "5", "3", "4", "6", "9", "8", "7", "%", "$", "\"", "/", "&", ":", "+"]
def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c for c in string if c in vocabs])
    return re.sub(r'[ ]+', ' ', string).strip()

In [25]:
processed_text = [preprocessing_text(t) for t in tqdm(texts)]

100%|██████████| 1403509/1403509 [00:29<00:00, 47381.15it/s]


In [26]:
audios[0], processed_text[0]

('/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20425643.mp3',
 'inilah dunia kecil')

In [27]:
with open('bahasa-asr-train.json', 'w') as fopen:
    json.dump({'X': audios, 'Y':processed_text}, fopen)

In [ ]:
# import malaya_speech

In [ ]:
# tokenizer = malaya_speech.subword.generate_tokenizer(processed_text, max_subword_length = 3)

In [ ]:
# malaya_speech.subword.save(tokenizer, 'transducer.subword')

In [ ]:
# tokenizer = malaya_speech.subword.load('transducer.subword')
# malaya_speech.subword.encode(tokenizer, 'i hate', add_blank = True)

In [ ]:
# malaya_speech.subword.decode(tokenizer, [0, 2, 133, 875])

In [ ]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 # print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [ ]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [ ]:
# shards = [{'split': 'train', 'shards': 1000}]

In [ ]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')